Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 20.792685
Training accuracy: 5.2%
Validation accuracy: 7.1%
Loss at step 100: 2.317365
Training accuracy: 71.9%
Validation accuracy: 71.4%
Loss at step 200: 1.874432
Training accuracy: 74.6%
Validation accuracy: 73.3%
Loss at step 300: 1.630759
Training accuracy: 75.9%
Validation accuracy: 74.3%
Loss at step 400: 1.463928
Training accuracy: 76.9%
Validation accuracy: 74.6%
Loss at step 500: 1.339851
Training accuracy: 77.5%
Validation accuracy: 74.9%
Loss at step 600: 1.243055
Training accuracy: 78.2%
Validation accuracy: 74.9%
Loss at step 700: 1.164729
Training accuracy: 78.9%
Validation accuracy: 74.9%
Loss at step 800: 1.099524
Training accuracy: 79.2%
Validation accuracy: 74.9%
Test accuracy: 82.4%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.552507
Minibatch accuracy: 10.9%
Validation accuracy: 9.8%
Minibatch loss at step 500: 1.094197
Minibatch accuracy: 78.1%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 1.494962
Minibatch accuracy: 78.1%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.793891
Minibatch accuracy: 78.9%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 0.964083
Minibatch accuracy: 81.2%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 1.071104
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 1.073887
Minibatch accuracy: 79.7%
Validation accuracy: 78.4%
Test accuracy: 85.6%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

# Plain Old Gradient Decent

In [32]:
# Try plain gradient decent
train_subset = 10000 # Setting up the size of the training set

# We start with an empty graph and then define the dataflow, and how the functions are chained to one another.
graph = tf.Graph() 
with graph.as_default():
    # Setting up the datasets using tf.constant()
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Setting up the weights and biases
    # Initiate weights with small numbers to avoid linear dependency in the matrix
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    w2 = tf.Variable(tf.truncated_normal([num_labels, num_labels]))
    
    # Not to worry too much about the biases and just make them all zeros
    biases = tf.Variable(tf.zeros([num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    
    # Setting up the output of the linear transform
    logits = tf.matmul(tf_train_dataset, weights) + biases # matrix operation -- input: features, output: logits
    
    # Setting up the ReLu layer
    relu_list = []
    
    for i in range(1):
        relu_list.append(tf.nn.relu(logits))
    
    # The second transformation
    lg2 = tf.matmul(np.sum(relu_list), w2) + b2
    
    # Setting up the loss function
    # Softmax and cross-entropy -- 1. input: logits, output: probabilies 2. input: probabilities, labels, output: entropy
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(lg2, tf_train_labels))
    
    # Now minimize the loss function we've defined above
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # As a result of the above line of code, the weights and biases have now been changed so that the loss is optimized.
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

In [33]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, axis=1) == np.argmax(labels, axis=1))) / predictions.shape[0]

# Now we start a session to run the data through our graph!
with tf.Session(graph=graph) as session:
    # Initiate all variables and run!
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
      # Run the computations. We tell .run() that we want to run the optimizer,
      # and get the loss value and the training predictions returned as numpy
      # arrays.
      _, l, predictions = session.run([optimizer, loss, train_prediction])
      if (step % 100 == 0):
        print('Loss at step %d: %f' % (step, l))
        print('Training accuracy: %.1f%%' % accuracy(
          predictions, train_labels[:train_subset, :]))
        # Calling .eval() on valid_prediction is basically like calling run(), but
        # just to get that one numpy array. Note that it recomputes all its graph
        # dependencies.
        print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 28.872709
Training accuracy: 10.3%
Validation accuracy: 9.4%
Loss at step 100: 1.538256
Training accuracy: 9.4%
Validation accuracy: 9.2%
Loss at step 200: 1.200962
Training accuracy: 11.4%
Validation accuracy: 11.5%
Loss at step 300: 1.060266
Training accuracy: 13.5%
Validation accuracy: 13.6%
Loss at step 400: 0.998236
Training accuracy: 14.7%
Validation accuracy: 14.6%
Loss at step 500: 0.888945
Training accuracy: 15.2%
Validation accuracy: 15.2%
Loss at step 600: 0.886977
Training accuracy: 15.8%
Validation accuracy: 15.6%
Loss at step 700: 0.818833
Training accuracy: 16.0%
Validation accuracy: 15.8%
Loss at step 800: 0.949608
Training accuracy: 16.2%
Validation accuracy: 15.9%
Test accuracy: 16.8%


# The Fancy SGD

In [82]:
H = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Training sets are no longer constants they are placeholders (not variables tho!)
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, H], stddev=np.sqrt(3.0/((image_size * image_size )+ H))))
    biases1 = tf.Variable(tf.zeros([H]))
    weights2 = tf.Variable(tf.truncated_normal([H, H], stddev=np.sqrt(3.0/(H + H))))
    biases2 = tf.Variable(tf.zeros([H]))
    weights3 = tf.Variable(tf.truncated_normal([H, num_labels], stddev=np.sqrt(3.0/(H + num_labels))))
    biases3 = tf.Variable(tf.zeros([num_labels]))
      
    # Training computation.
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
    logits = tf.matmul(hidden2, weights3) + biases3
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    h1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    h2_valid = tf.nn.relu(tf.matmul(h1_valid, weights2) + biases2)
    valid_logits = tf.matmul(h2_valid, weights3) + biases3
    valid_prediction = tf.nn.softmax(valid_logits)

    h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    h2_test = tf.nn.relu(tf.matmul(h1_test, weights2) + biases2)
    test_logits = tf.matmul(h2_test, weights3) + biases3
    test_prediction = tf.nn.softmax(test_logits)

In [83]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
  
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.405864
Minibatch accuracy: 8.6%
Validation accuracy: 19.1%
Minibatch loss at step 500: 0.327302
Minibatch accuracy: 89.8%
Validation accuracy: 86.4%
Minibatch loss at step 1000: 0.485497
Minibatch accuracy: 85.9%
Validation accuracy: 87.3%
Minibatch loss at step 1500: 0.239189
Minibatch accuracy: 93.0%
Validation accuracy: 88.5%
Minibatch loss at step 2000: 0.228997
Minibatch accuracy: 94.5%
Validation accuracy: 89.2%
Minibatch loss at step 2500: 0.282980
Minibatch accuracy: 91.4%
Validation accuracy: 89.3%
Minibatch loss at step 3000: 0.318090
Minibatch accuracy: 90.6%
Validation accuracy: 89.4%
Test accuracy: 95.2%
